Installing *yt-dlp* to download the image frame from the YouTube videos and *pyarrow* 

In [ ]:
!pip install yt-dlp
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing all the useful packages

In [ ]:
import yt_dlp
import os
from tqdm.notebook import tqdm
import cv2
import shutil
import time
import random
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
from datetime import timedelta

In [ ]:
# Importing drive to download or update datasets
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Setting Up Dataset


In [ ]:
# Hyperparameters

percentage_train_test = 10
percentage_bag_shots = 1
percentage_to_ignore = 10

zip_f_name = f'VideoClassificationDataset_{percentage_train_test}_{percentage_bag_shots}_{percentage_to_ignore}'
drive_dataset_folder = f'/content/drive/MyDrive/Datasets_VideoClassification/{zip_f_name}'

##Download frames

In [ ]:
dataset_path = Path('/content/VideoClassificationDataset')

# Download the .zip file
!wget --no-verbose https://github.com/gtoderici/sports-1m-dataset/archive/refs/heads/master.zip

# Extract it 
!unzip -qq -o '/content/master.zip' 
!rm '/content/master.zip

In [ ]:
DATA = {'train_partition.txt': {},
        'test_partition.txt': {}}

LABELS = []

train_dict = {}
test_dict = {}

path = '/content/sports-1m-dataset-master/original'

# Populate the DATA dictionary by reading the train and test files
for f in os.listdir(path):
    with open(path + '/' + f) as f_txt:
        lines = f_txt.readlines()
        for line in lines:
            splitted_line = line.split(' ')
            label_indices = splitted_line[1].rstrip('\n').split(',') 
            
            DATA[f][splitted_line[0]] = list(map(int, label_indices))

# Obtain the labels from the relative .txt file
with open('/content/sports-1m-dataset-master/labels.txt') as f_labels:
    LABELS = f_labels.read().splitlines()

In [ ]:
TRAIN = DATA['train_partition.txt']
TEST = DATA['test_partition.txt']
print('Original Train Test length: ', len(TRAIN), len(TEST))

# Sample a subset of percentage_train_test
TRAIN = dict(random.sample(list(TRAIN.items()), (len(TRAIN)*percentage_train_test)//100))
TEST = dict(random.sample(list(TEST.items()), (len(TEST)*percentage_train_test)//100))

print(f'Sampled {percentage_train_test} % of Train & Test datasets, updated length: ', len(TRAIN), len(TEST))

if not os.path.exists(dataset_path): os.makedirs(dataset_path)
if not os.path.exists(f'{dataset_path}/train'): os.makedirs(f'{dataset_path}/train')
if not os.path.exists(f'{dataset_path}/test'): os.makedirs(f'{dataset_path}/test')

Original Train Test length:  914491 218667
Sampled 10 % of Train & Test datasets, updated length:  91449 21866


In [ ]:
def extract_frames(capture, directory, idx_bag, start_frame, end_frame):
    '''
    PORPOUSE: Extract frames from a given youtube video

    TAKES:
      - capture: cv2.VideoCapture(url) variable, with url being the link to the video
      - directory: describe the saving directory
      - idx_bag: indicate the index of the actual bag of shots
      - start_frame: indicates the starting frame
      - end_frame: indicates the ending frame

    RETURNS:
      True or False depending the presence of errors
    '''

    count = start_frame

    # Set the next frame to download to 'count'
    capture.set(cv2.CAP_PROP_POS_FRAMES, count)
    os.makedirs(f'{directory}/bag_of_shots{str(idx_bag)}') # Create the relative directory

    # Download the frame until we do not reach the end_frame
    while count < end_frame:

        ret, frame = capture.read() # Read the frame

        if not ret: # In case there are errors, delete the aforementioned directory of bag of shots
            shutil.rmtree(f'{directory}/bag_of_shots{str(idx_bag)}')
            return False

        # Save the readed frame in the directory of bag of shots resizing it to be 178x178
        filename = f'{directory}/bag_of_shots{str(idx_bag)}/shot{str(count - start_frame)}.png'
        write_res = cv2.imwrite(filename, cv2.resize(frame, (178, 178), interpolation = cv2.INTER_AREA))
        if write_res:
            count += 1    # If there is no error I increment the count
        else:   # Otherwise I delete the all bag of shots returning Falses
            shutil.rmtree(f'{directory}/bag_of_shots{str(idx_bag)}')
            return False

    return True

In [ ]:
def video_to_frames(video_url, label_id, directory, percentage_of_bags):
    '''
    PORPOUSE: Determine the amount of frame and bag of shots we want to downloads in each video, and perform the download of them

    TAKES:
      - video_url: complete url link to a specific YouTubbe video
      - label_id: index label of the relative video
      - idx_bag: indicate the index of the actual bag of shots
      - percentage_of_bags: indicate the percentage of bags of shots that we want to download

    RETURNS: 
      ret_dictionary: A dictionary containing as keys the path to a specific bag of shots and as value the list of labels for the 
                      relative bag, the last element of this list represent the number of frames we have downloaded
    '''

    url_id = video_url.split('=')[1]
    path_until_url_id = f'{dataset_path}/{directory}/{url_id}'
    
    ret_dictionary = {}

    try:   

        # Setting up the dictionary options for yd-dlp
        ydl_opts = {
            'ignoreerrors': True,
            'quiet': True,
            'nowarnings': True,
            'ignorenoformatserror': True,
            'verbose':False,
            'cookies': '/content/all_cookies.txt',
            #https://stackoverflow.com/questions/63329412/how-can-i-solve-this-youtube-dl-429
        }
        ydl = yt_dlp.YoutubeDL(ydl_opts)
        info_dict = ydl.extract_info(video_url, download=False) # Extracting the video infromation

        if(info_dict is not None and  info_dict['fps'] >= 20):
            # I must have a least 20 frames per seconds since I take half of second bag of shots for every video

            formats = info_dict.get('formats', None)

            # Excluding the initial and final choosen percentage of each video to avoid noise
            video_length = info_dict['duration'] * info_dict['fps']

            shots = info_dict['fps'] // 2

            to_ignore = (video_length * percentage_to_ignore) // 100
            new_len = video_length - (to_ignore * 2)
            tot_stored_bags = ((new_len // shots) * percentage_of_bags) // 100   # ((total_possbile_bags // shots) * percentage_of_bags) // 100

            if tot_stored_bags == 0: tot_stored_bags = 1 # I take at least a bag of shots

            # Computing the skip rate between bags
            skip_rate_between_bags = (new_len - (tot_stored_bags * shots)) // (tot_stored_bags-1) if tot_stored_bags > 1 else 0

            chunks = [[to_ignore+(bag*(skip_rate_between_bags+shots)), to_ignore+(bag*(skip_rate_between_bags+shots))+shots] for bag in range(tot_stored_bags)]
            # Sequence of [[start_frame, end_frame], [start_frame, end_frame], [start_frame, end_frame], ...]

            format_id = {}
            for f in formats: format_id[f['format_id']] = f

            if '160' in list(format_id.keys()): # The downlaod is applyed only if the 144p format is available, 160 is the format_id for 144p
                video = format_id['160']
                url = video.get('url', None)
                if(video.get('url', None) != video.get('manifest_url', None)): # We move forward only in case the url is different from the manifest_url

                    if not os.path.exists(path_until_url_id): os.makedirs(path_until_url_id)
                    
                    capture = cv2.VideoCapture(url) # Initialize VideoCapture variable
                    valid_chunks = 0

                    for idx_bag, f in enumerate(chunks): # For each chunks

                        # In case the download of the bag of shots succedeed
                        if(extract_frames(capture, path_until_url_id, idx_bag, f[0], f[1])):
                          
                            l = np.array([label_id, shots])
                            valid_chunks += 1

                            ret_dictionary[f'{directory}/{url_id}/bag_of_shots{str(idx_bag)}'] = l.tolist() # Populate the new dictionary row

                    # In case we do not have downloaded any chunks delete the directory and all its content
                    if valid_chunks == 0: shutil.rmtree(path_until_url_id)

                    capture.release()

        return ret_dictionary


    except Exception as e:
        # If an exception rised delete the directory with all its content and return ret_dictionary
        if os.path.exists(path_until_url_id): shutil.rmtree(path_until_url_id)
        return ret_dictionary


In [ ]:
start_time = time.time()

count = 0

# Perform a parallel download
with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as pool:
    with tqdm(total = len(TRAIN.items()), leave=False, desc='Downloading Train Dataset') as progress: # Setting up the progress bar
        
        futures = [] # Array of features

        for url, label_id in TRAIN.items():
            future = pool.submit(video_to_frames, url, label_id, 'train', percentage_bag_shots) # Assign the download
            future.add_done_callback(lambda p: progress.update()) # Update the progrress bar
            futures.append(future) # Append the feature to the features array

        for future in futures:
            if len(future.result()) > 0: # In case the result of the featue is not empty
                train_dict.update(future.result()) # Append the result to the final dictionaty
                count += 1


print(f'--- Downloaded {count} videos frames in {(timedelta(seconds=(time.time() - start_time)))} (H:M:S:ms) ---')

In [ ]:
start_time = time.time()

count = 0

# Perform a parallel download
with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as pool:
    with tqdm(total = len(TEST.items()), leave=False, desc='Downloading Test Dataset') as progress: # Setting up the progress bar
        
        futures = [] # Array of features

        for url, label_id in TEST.items():
            future = pool.submit(video_to_frames, url, label_id, 'test', percentage_bag_shots) # Assign the download
            future.add_done_callback(lambda p: progress.update()) # Update the progrress bar
            futures.append(future) # Append the feature to the features array

        for future in futures:
            if len(future.result()) > 0: # In case the result of the featue is not empty
                test_dict.update(future.result()) # Append the result to the final dictionaty
                count += 1


print(f'--- Downloaded {count} videos frames in {(timedelta(seconds=(time.time() - start_time)))} (H:M:S:ms) ---')

## Save CSV and upload file Zip

In [ ]:
# Convert the dictionary to pandas DataFrame resetting the index
train_df = pd.DataFrame.from_dict(train_dict, orient='index', dtype=int).reset_index(level=0)
train_df = train_df.rename(columns={train_df.columns[-1]: 'shots'}) # Rename the last column as 'shots'
train_df.columns = train_df.columns.astype(str) # Setting the column type as string
train_df.to_parquet(f'{dataset_path}/train.parquet', index=True) # Generation the parquet file

# Convert the dictionary to pandas DataFrame resetting the index
test_df = pd.DataFrame.from_dict(test_dict, orient='index', dtype=int).reset_index(level=0)
test_df = test_df.rename(columns={test_df.columns[-1]: 'shots'}) # Rename the last column as 'shots'
test_df.columns = test_df.columns.astype(str) # Setting the column type as string
test_df.to_parquet(f'{dataset_path}/test.parquet', index=True) # Generation the parquet file

In [ ]:
shutil.make_archive('train', 'zip', f'{dataset_path}/train') # Create a train.zip file
shutil.make_archive('test', 'zip', f'{dataset_path}/test') # Create a test.zip file
os.mkdir(drive_dataset_folder) # Create remote drive folder

# Coping the .zip, .parquet and .txt file into the remote directory
shutil.copy('/content/sports-1m-dataset-master/labels.txt', drive_dataset_folder)
shutil.copy(f'{dataset_path}/train.parquet', drive_dataset_folder)
shutil.copy(f'{dataset_path}/test.parquet', drive_dataset_folder)
shutil.copy(f'{dataset_path}/train.zip', drive_dataset_folder)
shutil.copy(f'{dataset_path}/test.zip', drive_dataset_folder)

## Downloadload Dataset from Google Drive

In [ ]:
# With ZipFile I also download the .zip and at the same time unzipping it so I need the double capacity

dataset_path = Path('/content/VideoClassificationDataset')

# Unzip the train and test zip files in Colab Storage
for z in ['train', 'test']:
    with zipfile.ZipFile(f'{drive_dataset_folder}/{z}.zip', 'r') as zip_ref:
        for file in tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist()), desc=f'Unzipping {z}.zip'):
            zip_ref.extract(member=file, path=dataset_path)

# Copy the train and test .parquet and labels.txt in Colab Storage
shutil.copy(f'{drive_dataset_folder}/train.parquet', dataset_path)
shutil.copy(f'{drive_dataset_folder}/test.parquet', dataset_path)
shutil.copy(f'{drive_dataset_folder}/labels.txt', dataset_path)

print('All done')

Unzipping train.zip:   0%|          | 0/1592774 [00:00<?, ?it/s]

### Read labels.txt, train.parquet and test.parquet

In [ ]:
LABELS = []

train_dict = {}
test_dict = {}

# Obtain the list of labels
with open(f'{dataset_path}/labels.txt') as f_labels:
    LABELS = f_labels.read().splitlines()

train_df = pd.read_parquet(f'{dataset_path}/train.parquet') # Read the parquet train dataset
test_df = pd.read_parquet(f'{dataset_path}/test.parquet') # Read the parquet test dataset


for string, df, dic in zip(('train', 'test'), (train_df, test_df), (train_dict, test_dict)):
    for k,v in tqdm(df.T.items(), total=len(list(df.T.items())), desc=f'Populating {string} dictionary'):
        values = v.to_numpy()
        dic[values[0]] = list(values[1:]) # Populate the dictionary


print(f'\nSampled {percentage_train_test} % of Train & Test datasets')
print(f'Number of train bags of shots: {len(train_df)}')
print(f'Number of test bags of shots: {len(test_dict)}')

### Data Consistency Checking

In [ ]:
# Check the presence of empty or inconsistent directories

print(train_df[train_df['shots'] < 5][['index', 'shots']])
print(test_df[test_df['shots'] < 5][['index', 'shots']])

for idx, row in train_df[train_df['shots'] < 5].iterrows():
    print(row['index'], os.listdir(f"{dataset_path}/{row['index']}"))

for idx, row in test_df[test_df['shots'] < 5].iterrows():
    print(row['index'], os.listdir(f"{dataset_path}/{row['index']}"))

to_del_train = []
for idx, row in train_df.iterrows():
    for shot_id in range(row['shots']):
        if f'shot{shot_id}.png' not in os.listdir(f"{dataset_path}/{row['index']}"):
            print(row['index'], os.listdir(f"{dataset_path}/{row['index']}"))
            to_del_train.append(idx)
            break

to_del_test = []
for idx, row in test_df.iterrows():
    for shot_id in range(row['shots']):
        if f'shot{shot_id}.png' not in os.listdir(f"{dataset_path}/{row['index']}"):
            print(row['index'], os.listdir(f"{dataset_path}/{row['index']}"))
            to_del_test.append(idx)
            break


In [ ]:
# Throw away those directory with correspondant row of dictionary and dataframe

to_delete_train = train_df[train_df['shots'] < 5]
to_delete_train = pd.concat([to_delete_train, train_df.iloc[to_del_train]], axis=0)
train_df.drop(to_delete_train.index, axis=0, inplace=True)
train_df.reset_index()

for idx, to_d in to_delete_train.iterrows():
    del train_dict[to_d['index']]
    shutil.rmtree(f"{dataset_path}/{to_d['index']}")


to_delete_test = test_df[test_df['shots'] < 5]
to_delete_test = pd.concat([to_delete_test, test_df.iloc[to_del_test]], axis=0)
test_df.drop(to_delete_test.index, axis=0, inplace=True)
test_df.reset_index()

for idx, to_d in to_delete_test.iterrows():
    del test_dict[to_d['index']]
    shutil.rmtree(f"{dataset_path}/{to_d['index']}")

# Neural Networks Part

In [ ]:
import requests
import os
import random
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from timeit import default_timer as timer 
import copy

from google.colab import widgets

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms

torch.autograd.set_detect_anomaly(True)

print(f'PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}')


In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device # Setup the device type

In [ ]:
# Get the train and test path
train_dir = dataset_path / 'train'
test_dir = dataset_path / 'test'

## Initial analysis

In [ ]:
'''
PORPOUSE: Obtaining the label name and index of a specific image

TAKES:
  - dictionary: is the dictionary from where we have to extract the ids
  - path: describe the path of the specific bag of shots

RETURNS: Pairs of label index and name
'''

def get_label_id_name(dictionary, path):
    id = np.where(np.asarray(dictionary[path]) == 1)[0]
    return id, LABELS[id] 

In [ ]:
'''
PORPOUSE: Plot a random dataset image with its informations

TAKES:
  - dictionary: is the dictionary from where we have to extract the ids
  - image_path_list: describe the path where we have to sample a image

RETURNS: Pillow image
'''

def plot_random_image(dictionary, image_path_list):
    random_image_path = random.choice(image_path_list) # Sample a image

    path = os.path.relpath(random_image_path.parent, dataset_path) # Get the relative path
    
    image_class_name, image_class_id = get_label_id_name(dictionary, path)
    img = Image.open(random_image_path) # Open the image

    print(f'Random image path: {random_image_path}')
    print(f'Image classes id: {image_class_id}')
    print(f'Image classes name: {image_class_name}')
    print(f'Image height: {img.height}') 
    print(f'Image width: {img.width}')

    return img

plot_random_image(train_dict, list(train_dir.glob('*/*/*.png')))

Transforms

In [ ]:
# Setting up the train and test transofms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.4588,0.4588,0.4588], std=[0.4588,0.4588,0.4588])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.4588,0.4588,0.4588], std=[0.4588,0.4588,0.4588])
])

## From Dataset to Dataloader

Dataset creation

In [ ]:
# VideoDataset object 

class VideoDataset(Dataset):
    def __init__(self, df, transform = None, t = 'single'):
        self.df = df
        self.transform = transform
        self.t = t
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        if self.transform is None: self.transform = transforms.ToTensor()
        images_path = self.df.iloc[index, 0]

        shots = self.df.iloc[index, -1] # Get the number of frames of a bag of shots
        # Each bag is half second frames

        if self.t == 'single':
            # I want only the central frame
            images = self.transform(Image.open(f'{dataset_path}/{images_path}/shot{shots//2}.png'))

        if self.t == 'early':
            # I want the 5 middle frames
            images = np.array([self.transform(Image.open(f'{dataset_path}/{images_path}/shot{idx}.png')).numpy() for idx in range((shots//2)-2,(shots//2)+3)])
        
        if self.t == 'late':
            # I want the first and last frames
            images = np.array([self.transform(Image.open(f'{dataset_path}/{images_path}/shot0.png')).numpy(), self.transform(Image.open(f'{dataset_path}/{images_path}/shot{shots-1}.png')).numpy()])
        
        if self.t == 'slow':
            # I want the 10 middle frames
            if shots%10 == 0:
                images = np.array([self.transform(Image.open(f'{dataset_path}/{images_path}/shot{idx}.png')).numpy() for idx in range((shots//2) - 5, (shots//2) + 5)])
            else:
                images = np.array([self.transform(Image.open(f'{dataset_path}/{images_path}/shot{idx}.png')).numpy() for idx in range((shots%10) - (shots%10)//2, shots-(shots%10)//2)])

        y_label = torch.from_numpy(self.df.iloc[index, 1:-1].to_numpy().astype(float)) # Get the labels

        if self.t != 'single': images = torch.from_numpy(images) # Convert the numpy image to tensor in case I want the central frame

        return images, y_label, images_path

Split the training set into train and validation set

In [ ]:
def spit_train(train_data, perc_val_size):
    '''
    PORPOUSE: Splot the dataset in train and validation set

    TAKES:
      - train_data: dataset to split
      - perc_val_size: percentage of split

    RETURNS: train dataset and validation dataset
    '''
    
    train_size = len(train_data)
    val_size = int((train_size * perc_val_size) // 100)
    train_size -= val_size

    return random_split(train_data, [int(train_size), int(val_size)]) #train_data, val_data 

Dataset definition

In [ ]:
# Let's set the validations set to the 20% of the train dataset

# Dataset for Single Frame
train_data_single, val_data_single = spit_train(VideoDataset(df=train_df, transform=train_transform, t='single'), 20)
test_data_single = VideoDataset(df=test_df, transform=test_transform, t='single')

# Dataset for Multi Frame - Early Fusion
train_data_early, val_data_early = spit_train(VideoDataset(df=train_df, transform=train_transform, t='early'), 20)
test_data_early = VideoDataset(df=test_df, transform=test_transform, t='early')

# Dataset for Multi Frame - Late Fusion
train_data_late, val_data_late = spit_train(VideoDataset(df=train_df, transform=train_transform, t='late'), 20)
test_data_late = VideoDataset(df=test_df, transform=test_transform, t='late')

# Dataset for Multi Frame - Slow Fusion
train_data_slow, val_data_slow = spit_train(VideoDataset(df=train_df, transform=train_transform, t='slow'), 20)
test_data_slow = VideoDataset(df=test_df, transform=test_transform, t='slow')

### Random Image from the train and test dataloader for Single Frame model

In [ ]:
def random_image_from_dataloader(dt, dic):
    '''
    PORPOUSE: Plot a random image from a given dataset

    TAKES:
      - dt: dataset where sample a image
      - dic: dictionary to which the image belongs

    RETURNS: train dataset and validation dataset
    '''
    
    rnd = random.choice(dt)
    img, label, path = rnd
    label_id, label_name = get_label_id_name(dic, path)

    print(f'Image tensor:\n{img}')
    print(f'Image shape: {img.shape}')
    print(f'Image datatype: {img.dtype}')
    print(f'Image label ids: {label_id}')
    print(f'Image label names: {label_name}')
    print(f'Label datatype: {type(label)}')
    plt.imshow(img.permute(1, 2, 0).numpy())
    plt.axis(False)

In [ ]:
random_image_from_dataloader(train_data_single, train_dict) # Random image from the train dataset

In [ ]:
random_image_from_dataloader(test_data_single, test_dict) # Random image from the test dataset


### Turn loaded images into DataLoader's

In [ ]:
# Turn train and test satasets into DataLoaders
BATCH_SIZE = 32 # <---------------------------- TODO: tune this paramenter
NUM_WORKERS = os.cpu_count()

In [ ]:
def generate_dataloaders(train_data, val_data, test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
    '''
    PORPOUSE: generate the train vaiadtion and test dataloaders

    TAKES:
      - train_data: train dataset
      - val_data: validation dataset
      - test_data: test dataset
      - batch_size: specify how many bags of shots a bach must have
      - num_workers: specify how many workers will work on the dataloaders creation 

    RETURNS: train, val and test dataloaders
    '''
    
    train_dl = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, num_workers = NUM_WORKERS, shuffle = True)
    val_dl = DataLoader(dataset = val_data, batch_size = BATCH_SIZE, num_workers = NUM_WORKERS, shuffle = True)
    test_dl = DataLoader(dataset = test_data, batch_size = BATCH_SIZE, num_workers = NUM_WORKERS, shuffle = False)

    return train_dl, val_dl, test_dl

## Models definitions

In [ ]:
# Defining the main custom Alexnet that ereditate the fucntion of nn.Module

class AlexNet(nn.Module):
    def __init__(self, in_channels, stream_type=None, t_frames=[1,1,1]):
        # stream_type: none, "context", "fovea"
        # None is used for considering the cnn with spartial fusion informations
        super().__init__()

        # input size nomultiresulution : (b x 3 x 170 x 170)
        # input size multiresulution : (b x 3 x 178 x 178) take into account the video classifier -> which becomes 89 x 89 in both streams

        self.stream_type = stream_type
        self.fovea = transforms.Compose([transforms.CenterCrop((89, 89))])
        self.context = transforms.Compose([transforms.Resize((89, 89))])
        self.transform = transforms.Compose([transforms.Resize((170, 170))])
        self.t_frames = t_frames

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels*self.t_frames[0], 96, kernel_size=11, stride=3, padding=2),  
            # CORRECT
            # NO-MULTIRES -> in: (b x 96 x 170 x 170) -> (b x 96 x 55 x 55)
            # MULTIRES -> in: (b x 96 x 89 x 89) -> (b x 96 x 28 x 28)
            nn.ReLU(inplace=True),

            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.5, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # CORRECT
            # NO-MULTIRES -> in: (b x 96 x 55 x 55) -> (b x 96 x 28 x 28)
            # MULTIRES -> in: (b x 96 x 28 x 28) -> (b x 96 x 14 x 14)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(96*self.t_frames[1], 256, kernel_size=5, stride=1, padding=2), 
            # CORRECT
            # NO-MULTIRES -> in: (b x 96 x 28 x 28) -> (b x 256 x 28 x 28)
            # MULTIRES -> in: (b x 96 x 14 x 14) -> (b x 256 x 14 x 14)
            nn.ReLU(inplace=True),

            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.5, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            # CORRECT
            # NO-MULTIRES -> in: (b x 256 x 28 x 28) -> (b x 256 x 14 x 14)
            # MULTIRES -> in: (b x 256 x 14 x 14) -> (b x 256 x 7 x 7)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(256*self.t_frames[2], 384, kernel_size=3, stride=1, padding=1), 
            # CORRECT
            # NO-MULTIRES -> in: (b x 256 x 14 x 14) -> (b x 384 x 14 x 14)
            # MULTIRES -> in: (b x 256 x 7 x 7) -> (b x 384 x 7 x 7)
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            # CORRECT
            # NO-MULTIRES -> in: (b x 384 x 14 x 14) -> (b x 384 x 14 x 14)
            # MULTIRES -> in: (b x 384 x 7 x 7) -> (b x 384 x 7 x 7)
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            # CORRECT
            # NO-MULTIRES -> in: (b x 384 x 14 x 14) -> (b x 256 x 14 x 14)
            # MULTIRES -> in: (b x 384 x 7 x 7) -> (b x 256 x 7 x 7)
            nn.ReLU(inplace=True)
        )

        self.MaxPool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # CORRECT
        # NO-MULTIRES -> in: (b x 256 x 14 x 14) -> (b x 256 x 7 x 7)

        self.init_bias()  # initialize bias -> CHECK IF IT MAKE SENSE

    def init_bias(self):
        for block in [self.conv1, self.conv2, self.conv3]:
            for layer in block:
                if isinstance(layer, nn.Conv2d):
                    nn.init.normal_(layer.weight, mean=0, std=0.01)
                    nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.conv2[0].bias, 1)
        nn.init.constant_(self.conv3[2].bias, 1)
        nn.init.constant_(self.conv3[4].bias, 1)


    def forward(self, x):
        if self.stream_type != None:
            x = self.fovea(x) if self.stream_type == 'fovea' else self.context(x)
            return self.conv3(self.conv2(self.conv1(x)))
        else: 
            x = self.transform(x)
            return self.MaxPool(self.conv3(self.conv2(self.conv1(x))))

### Single frame CNNs:
1. Single-Fame (custom AlexNet)
2. Single-Fame Fovea Only
3. Single-Fame Context Only
4. Single-Fame + Multires



In [ ]:
# The single frame CNN, includes the three CNNs from the above list

class SingleResCNN(nn.Module):
    def __init__(self, CNN, num_classes):
        super(SingleResCNN, self).__init__()
        self.CNN = CNN

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=(256 * 7 * 7), out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=num_classes),
        )

        
    def forward(self, x):
        x = self.CNN(x)
        x = x.reshape(x.shape[0], -1)
        return self.classifier(x)

In [ ]:
# Defining the single frame multiresolution CNN 

class MultiResCNN(nn.Module):
    def __init__(self, AlexNet_fovea, AlexNet_context, num_classes):
        super(MultiResCNN, self).__init__()
        self.AlexNet_fovea = AlexNet_fovea
        self.AlexNet_context = AlexNet_context

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=(512 * 7 * 7), out_features=4096), # 512 since it takes the double of the infrormations
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        
    def forward(self, x):
        x1 = self.AlexNet_fovea(x.clone()) # Takes the fovea strea
        x2 = self.AlexNet_context(x.clone()) # Takes the contxt stream
        x = torch.cat((x1, x2), dim=1)
        x = x.reshape(x.shape[0], -1)
        return self.classifier(x)

### Multi Frames CNNs
1. Late Fusion
2. Early Fusion
3. Slow Fusion

In [ ]:
# Late Fusion Model

class LateFusionCNN(nn.Module):
    def __init__(self, AlexNet_1, AlexNet_2, num_classes):
        super(LateFusionCNN, self).__init__()
        self.AlexNet_1 = AlexNet_1 # Two Separates AlexNet
        self.AlexNet_2 = AlexNet_2

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=(512 * 7 * 7), out_features=4096),  # 512 since it takes the double of the infrormations
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        
    def forward(self, x):
        initial = torch.empty(x.shape[0], 3, 178, 178).to(device)
        final = torch.empty(x.shape[0], 3, 178, 178).to(device)
        
        for i in range(x.shape[0]):
            initial[i] = x[i][0]
            final[i] = x[i][1]

        initial = self.AlexNet_1(initial)
        final = self.AlexNet_2(final)
        x = torch.cat((initial, final), dim=1)
        x = x.reshape(x.shape[0], -1)
        return self.classifier(x)

In [ ]:
# Early Fusion Model

class EarlyFusionCNN(nn.Module):
    def __init__(self, AlexNet, num_classes):
        super(EarlyFusionCNN, self).__init__()
        self.AlexNet = AlexNet

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=(256 * 7 * 7), out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        
    def forward(self, x):
        x = x.reshape(x.shape[0], x.shape[1]*3, 178, 178)
        x = self.AlexNet(x)
        x = x.reshape(x.shape[0], -1)
        return self.classifier(x)

In [ ]:
# Sow Fusion Model

class SlowFusionCNN(nn.Module):
    def __init__(self, AlexNet, num_classes):
        super(SlowFusionCNN, self).__init__()
        self.AlexNet = AlexNet

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=(256 * 7 * 7), out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        
    def forward(self, x):
        bag = torch.empty(4,x.shape[0],3*self.AlexNet.t_frames[0],178,178).to(device)
        
        for idx in range(bag.shape[0]): # 0 - 4
            for i in range(x.shape[0]): # 0 - batch_size
                bag[idx][i] = x[i][(idx) : (idx+4)].reshape(3*self.AlexNet.t_frames[0],178,178)

        rconv2_2 = torch.cat((self.AlexNet.conv2( # Second layer
            torch.cat((self.AlexNet.conv1(bag[0]), self.AlexNet.conv1(bag[1])), dim=1) # First layer
        ), self.AlexNet.conv2( # Second layer
            torch.cat((self.AlexNet.conv1(bag[2]), self.AlexNet.conv1(bag[3])), dim=1) # First layer
        )), dim=1)

        x = self.AlexNet.MaxPool(self.AlexNet.conv3(rconv2_2)) # Third layer

        x = x.reshape(x.shape[0], -1)
        return self.classifier(x)

### Train and Evaluate Architecture

In [ ]:
# CNN Architecture to perform the Train and Evaluation steps saving the results

class CNN_Architecture():

    def __init__(self, model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, 
        val_dataloader: torch.utils.data.DataLoader, optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module, score_fn, scheduler: torch.optim.Optimizer, device: torch.device,
        save_check = False, load_check_train = False, load_check_evaluate = False):

        self.model = model.to(device)
        self.optimizer = optimizer
        self.train_dataloader = train_dataloader
        self.loss_fn = loss_fn
        self.val_dataloader = val_dataloader
        self.score_fn = score_fn
        self.scheduler = scheduler
        self.device = device
        self.save_check = save_check
        self.load_check_train = load_check_train
        self.load_check_evaluate = load_check_evaluate
        if self.model.__class__.__name__ == 'SingleResCNN':
            self.model_name = f'{self.model.__class__.__name__}-Stream_Type_{self.model.CNN.stream_type}'
        else: 
            self.model_name = self.model.__class__.__name__
        self.best_checkpoint_filename = f'/content/drive/MyDrive/checkpoints/best_checkpoints/{self.model_name}_checkpoint.pth.tar'
        self.last_checkpoint_filename = f'/content/drive/MyDrive/checkpoints/last_checkpoints/{self.model_name}_checkpoint.pth.tar'



    def __save_checkpoint(self, results, checkpoint_filename, best_train_loss = None, best_train_accuracy = None):
        '''
        PORPOUSE: save the a checkpoint model

        TAKES:
          - results: dictionary containing all the results up that epoch
          - checkpoint_filename: google drive path where to save the checkpoint 
          - best_train_loss: best train loss to save
          - best_train_accuracy: best train accuracy score to save

        RETURNS: None
        '''

        print(f'=> Saving Checkpoint to {checkpoint_filename.split("/")[5]}')
        checkpoint = {'state_dict': self.model.state_dict(), 'optimizer': self.optimizer.state_dict(), 'scheduler': self.scheduler.state_dict(),
                      'results': results, 'best_train_loss': best_train_loss, 'best_train_accuracy': best_train_accuracy}
        torch.save(checkpoint, checkpoint_filename)
        print(' DONE\n')


    
    def __load_best_checkpoint(self):
        '''
        PORPOUSE: Load the best chekpoint model

        RETURNS: None
        '''

        print('=> Loading Best Checkpoint')
        checkpoint = torch.load(self.best_checkpoint_filename)
        self.model.load_state_dict(checkpoint['state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        print(' DONE\n')


    def __load_last_checkpoint(self):
        '''
        PORPOUSE: Load the last chekpoint model

        RETURNS: results fit history
        '''

        print('=> Loading Last Checkpoint')
        checkpoint = torch.load(self.best_checkpoint_filename)
        self.model.load_state_dict(checkpoint['state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        print(' DONE\n')
        return checkpoint['results'], checkpoint['best_train_loss'], checkpoint['best_train_accuracy']

    
    def evaluate(self, val_dataloader: torch.utils.data.DataLoader, epoch = 0, epochs = 1):
        '''
        PORPOUSE: Perform the model evaluation / testing

        TAKES:
          - val_dataloader: checkpoint to load
          - epoch / epochs

        RETURNS: dictionary containing the model name, loss and socore
        '''

        val_loss, val_accuracy = 0, 0

        if self.load_check_evaluate: self.__load_best_checkpoint() # If the flag is true I load the best checkpoint

        self.model.eval() # Evaluation phase

        pbar = tqdm(enumerate(val_dataloader), total = len(val_dataloader), leave=False) # Initialize the progress bar

        with torch.inference_mode(): # Allow inference mode
            for batch_idx, (images, label, _) in pbar:
                images, label = images.to(self.device), label.to(self.device) # Move the images and labels into the device

                outputs = self.model(images) # Get the model output

                loss = self.loss_fn(outputs, label) # Get the loss

                accuracy = self.score_fn(outputs, label).item() # Perform the score

                # Increment the statistics
                val_loss += loss.item()
                val_accuracy += accuracy

                # Update the progress bar
                if epoch > 0: pbar.set_description(f'{self.model_name} EVALUATION Epoch [{epoch + 1} / {epochs}]')
                else: pbar.set_description(f'{self.model_name} TESTING')
                pbar.set_postfix(loss = loss.item(), accuracy = accuracy)
              
            val_loss /= len(val_dataloader) # Calculate the final loss
            val_accuracy /= len(val_dataloader) # Calculate the final score


        return { 'model_name': self.model_name,
                'model_loss': val_loss,
                'model_accuracy': val_accuracy }


    
    def fit(self, epochs: int):
        '''
        PORPOUSE: Perform the model traing

        TAKES:
          - epochs: number of times that our model will see the data

        RETURNS: dictionary of results containing model name and history of results for each epoch
        '''

        results = { 'train_loss': [], 'train_v': [], 'val_loss': [], 'val_accuracy': [] }
        best_train_loss, best_train_v = float('inf'), float('-inf')

        if self.load_check_train:
            # If the flag is true I load the last checkpoint, importing the previous results history whereas the best loss and accuracy score
            results, best_train_loss, best_train_v = self.__load_last_checkpoint() 

        for epoch in range(len(results['train_loss']), epochs):
            train_loss, train_accuracy = 0, 0

            pbar = tqdm(enumerate(self.train_dataloader), total = len(self.train_dataloader), leave=False) # Initialize the progress bar
            
            for batch_idx, (images, label, _) in pbar:

                self.model.train() # Training phase

                # zero_grad -> backword -> step

                self.optimizer.zero_grad()
                images, label = images.to(self.device), label.to(self.device) # Move the images and labels into the device
                
                outputs = self.model(images) # Get the model output

                loss = self.loss_fn(outputs, label) # Get the loss

                loss.backward() # Backword step
                self.optimizer.step()

                train_loss += loss.item()
                accuracy = self.score_fn(outputs, label).item() # Perform the score

                train_accuracy += accuracy


                # Update the progress bar
                pbar.set_description(f'{self.model_name} TRAIN Epoch [{epoch + 1} / {epochs}]')
                pbar.set_postfix(loss = loss.item(), accuracy = accuracy)


            train_loss /= len(self.train_dataloader) # Calculate the final loss
            train_accuracy /= len(self.train_dataloader) # Calculate the final score


            self.scheduler.step(train_loss)
            

            # Validation phase
            model_name, val_loss, val_accuracy = (self.evaluate(self.val_dataloader, epoch, epochs)).values()

            # Append the results of the current epoch
            results['train_loss'].append(train_loss)
            results['train_accuracy'].append(train_accuracy)
            results['val_loss'].append(val_loss)
            results['val_accuracy'].append(val_accuracy)

            print('Epoch [{}], train_loss: {:.6f}, train_accuracy: {:.6f}, val_loss: {:.6f}, val_accuracy: {:.6f} \n'.format(
                  epoch + 1, train_loss, train_accuracy, val_loss, val_accuracy))
            

            if(self.save_check): 
                self.__save_checkpoint(results, self.last_checkpoint_filename, train_loss, train_accuracy) # Save a checkpoint 
                if(train_loss < best_train_loss and train_accuracy > best_train_accuracy): # Save a checkpoint if the actual results are the best one
                    self.__save_checkpoint(results, self.best_checkpoint_filename)
                    best_train_loss,best_train_accuracy  = train_loss, train_accuracy

        return {'model_name': self.model_name, 'results': results}

    
    
    def evaluate_and_plot_image(self, images_tensor, class_names, transform=None, mean=[0.4588,0.4588,0.4588], std=[0.4588,0.4588,0.4588]):
        '''
        PORPOUSE: Perform the evaluation of an image and plot it

        TAKES:
          - image_tensor: tensor representing the image 
          - class_names: labels name of our image
          - transform
          - mean
          - std

        RETURNS: None
        '''

        topk = 3 # Number of top k labels that we want to see

        if transform is not None: image_transform = transform
        else: image_transform = transforms.Compose([ transforms.Normalize(mean=mean, std=std) ])
        
        self.model.to(self.device) # Move the model to device

        self.model.eval() # Evauation phase
        
        with torch.inference_mode(): # Allow inference mode
            transformed_image = image_transform(images_tensor).unsqueeze(dim=0)
            targets_image_pred = self.model(transformed_image.to(self.device)) # Get the model output

        target_image_pred_probs = torch.sigmoid(targets_image_pred).to('cpu')
        top_k_probs_labels = torch.topk(target_image_pred_probs, k=topk, dim=1) # Get the probability of the top K labels

        label_pred_names = [LABELS[int(lab.numpy())]for lab in top_k_probs_labels.indices[0]] # Get the name of the top K labels


        # Images plot

        print(f'True Labels:\t{class_names}\nTop {topk} Pred:\t{label_pred_names}\nTop {topk} Prob:\t{top_k_probs_labels.values[0].numpy().tolist()}')


        if(len(images_tensor.shape) == 3):
            plt.figure(figsize=(5,5))
            plt.title(f"Model: {self.model_name}")
            plt.imshow(images_tensor.permute(1, 2, 0).numpy())
            plt.axis(False)
            plt.show()
        else:
            ids_frames = []
            if images_tensor.shape[0] == 2: ids_frames = [1, -1]
            elif images_tensor.shape[0] == 5: ids_frames = np.arange(5).tolist()
            else: ids_frames = np.arange(10).tolist()

            tb = widgets.TabBar([f'Frame {str(i)}' for i in ids_frames])

            for img in range(0, images_tensor.shape[0]):
                with tb.output_to(img, select=(img == 0)):
                    plt.title(f"Model: {self.model_name} - Frame: {ids_frames[img]}")
                    plt.imshow(images_tensor[img].permute(1, 2, 0).numpy())
                    plt.axis(False)
                    plt.show()
                    
                  

In [ ]:
'''
PORPOUSE: Perform Accuracy Score

TAKES:
  - outputs: model output
  - labels: ground truth

RETURNS: accuracy score
'''

def accuracy_score(outputs, labels):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

# Let's run all!

In [ ]:
NUM_EPOCHS = 5
n_classes = len(LABELS)

In [ ]:
def plot_loss_curves(results_info):
    '''
    PORPOUSE: Plot Loss and Score curves

    TAKES:
      - results_info: dictionary containing the results and model name

    RETURNS: None
    '''

    res = results_info['results']
    epochs = range(len(res['train_loss']))

    fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (15, 7))

    plt.title(results_info['model_name'])

    ax[0].plot(epochs, res['train_loss'], label = 'train_loss')
    ax[0].plot(epochs, res['val_loss'], label = 'val_loss')
    ax[0].set_title('Loss - Epochs')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[0].grid()
    ax[0].legend()

    ax[1].plot(epochs, res['train_accuracy'], label = 'train_accuracy_score')
    ax[1].plot(epochs, res['val_accuracy'], label = 'val_accuracy_score')
    ax[1].set_title('Accuracy Score - Epochs')
    ax[1].set_ylabel('Accuracy Score')
    ax[1].set_xlabel('Epochs')
    ax[1].grid()
    ax[1].legend()

    plt.show()

In [ ]:
def train_evaluate(model, test, epochs=NUM_EPOCHS):
    '''
    PORPOUSE: Perform the training and evaluation plotting the results

    TAKES:
      - model: CNN model to use
      - test: test dataloader
      - EPOCHS: numer of times that our model will see the data

    RETURNS: Train history and results from test evaluation
    '''

    start_time = timer()
    history = model.fit(NUM_EPOCHS)  # Train the model, it returns {'model_name': model_name, 'results': results}
    end_time = timer()

    print(f'Total training time: {end_time-start_time:.3f} seconds')

    plot_loss_curves(history) # Compare the results between train and validation set

    start_time = timer()
    result = model.evaluate(test) # Evaluate the model in the Tran dataloader
    # It returns {'model_name': model_name, 'model_loss': val_loss.item(), 'model_accuracy': val_accuracy.item()}
    end_time = timer()


    print(f'Total evaluation time: {end_time-start_time:.3f} seconds\n')
    print(f"TEST Results for {result['model_name']} -> loss: {result['model_loss']} accuracy-score: {result['model_accuracy']}")
    
    return (history, result)

In [ ]:
def test_single_image(dl, dic, cnn_arc):
    '''
    PORPOUSE: Perform the evaluation given a single image

    TAKES:
      - dl: image dataloader
      - dic: image dictionary
      - cnn_arc: CNN model

    RETURNS: None 
    '''

    rnd = random.choice(dl)
    img, label, path = rnd
    _, label_name = get_label_ids_names(dic, path)
    cnn_arc.evaluate_and_plot_image(img, label_name)

In [ ]:
def calculate_pos_weights(train_df, test_df, n_classes):
    '''
    PORPOUSE: Calculate the class weights since we have an imbalance dataset

    TAKES:
      - train_ds: train dataframe
      - test_ds: test dataframe
      - n_classes: number of classes

    RETURNS: np.array of weights
    '''

    col = list(map(str, range(n_classes)))

    count_train = train_df[col].sum(axis=0).to_numpy()
    count_test = test_df[col].sum(axis=0).to_numpy()
    class_counts = count_train + count_test

    l = len(train_df) + len(test_df)

    pos_weights = np.ones_like(class_counts)
    neg_counts = [l - pos_count for pos_count in class_counts]


    for cdx, (pos_count, neg_count) in enumerate(zip(class_counts.tolist(),  neg_counts)):
        pos_weights[cdx] = neg_count / (pos_count + 1e-5) # asserts also division by zero since we add 1e-5

    return torch.as_tensor(pos_weights, dtype=torch.float).to(device)

weights = calculate_pos_weights(train_df, test_df, n_classes)
#weights

Dataloaders definitons

In [ ]:
train_dl_single, val_dl_single, test_dl_single = generate_dataloaders(train_data_single, val_data_single, test_data_single)
train_dl_late, val_dl_late, test_dl_late = generate_dataloaders(train_data_late, val_data_late, test_data_late)
train_dl_early, val_dl_early, test_dl_early = generate_dataloaders(train_data_early, val_data_early, test_data_early)
train_dl_slow, val_dl_slow, test_dl_slow = generate_dataloaders(train_data_slow, val_data_slow, test_data_slow)

Defining the container of the experiments

In [ ]:
experiments_results = []

## Single Frame Model CNN

In [ ]:
AN_single = AlexNet(in_channels=3) # Extractor
Single_CNN = SingleResCNN(AN_single, num_classes=n_classes) # CNN model

optimizer_single = torch.optim.SGD(params=Single_CNN.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-5) # Optimizer 
scheduler_single = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_single, factor=0.1, patience=2, verbose=True) # Scheduler

# Architecture definition
CNN_arch_single = CNN_Architecture(model = Single_CNN, 
             train_dataloader = train_dl_single,
             val_dataloader = val_dl_single,
             optimizer = optimizer_single,
             loss_fn = nn.CrossEntropyLoss(weight=weights),
             score_fn = accuracy_score,
             scheduler = scheduler_single,
             device = device,
             save_check = True,
             load_check_train = False,
             load_check_evaluate = False)

In [ ]:
experiments_results.append(train_evaluate(CNN_arch_single, test_dl_single)) # Run train and evaluation

In [ ]:
test_single_image(test_data_single, test_dict, CNN_arch_single) # Test in a single random image

## Single Frame Model Fovea CNN

In [ ]:
AN_Fovea = AlexNet(in_channels=3, stream_type='fovea') # Extractor
Fovea_CNN = SingleResCNN(AN_Fovea, num_classes=n_classes) # CNN model

optimizer_fovea = torch.optim.SGD(params=Fovea_CNN.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-5) # Optimizer 
scheduler_fovea = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_fovea, factor=0.1, patience=2, verbose=True) # Scheduler

# Architecture definition
CNN_arch_fovea = CNN_Architecture(model = Fovea_CNN, 
             train_dataloader = train_dl_single,
             val_dataloader = val_dl_single,
             optimizer = optimizer_fovea,
             loss_fn = nn.CrossEntropyLoss(weight=weights),
             score_fn = accuracy_score,
             scheduler = scheduler_fovea,
             device = device,
             save_check = True,
             load_check_train = True,
             load_check_evaluate = False)

In [ ]:
experiments_results.append(train_evaluate(CNN_arch_fovea, test_dl_single)) # Run train and evaluation

In [ ]:
test_single_image(test_data_single, test_dict, CNN_arch_fovea) # Test in a single random image

## Single Frame Model Context CNN

In [ ]:
AN_Context = AlexNet(in_channels=3, stream_type='context') # Extractor
Context_CNN = SingleResCNN(AN_Context, num_classes=n_classes) # CNN model

optimizer_context = torch.optim.SGD(params=Context_CNN.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-5) # Optimizer 
scheduler_context = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_context, factor=0.1, patience=2, verbose=True) # Scheduler

# Architecture definition
CNN_arch_context = CNN_Architecture(model = Context_CNN, 
             train_dataloader = train_dl_single,
             val_dataloader = val_dl_single,
             optimizer = optimizer_context,
             loss_fn = nn.CrossEntropyLoss(weight=weights),
             score_fn = accuracy_score,
             scheduler = scheduler_context,
             device = device,
             save_check = True,
             load_check_train = True,
             load_check_evaluate = False)

In [ ]:
experiments_results.append(train_evaluate(CNN_arch_context, test_dl_single)) # Run train and evaluation

In [ ]:
test_single_image(test_data_single, test_dict, CNN_arch_context) # Test in a single random image

## Single Frame Model Multiresolution CNN

In [ ]:
AN_Multi_Fovea, AN_Multi_context = AlexNet(in_channels=3, stream_type='fovea'), AlexNet(in_channels=3, stream_type='context') # Extractors
Multi_CNN = MultiResCNN(AN_Multi_Fovea, AN_Multi_context, num_classes=n_classes) # CNN model

optimizer_multi = torch.optim.SGD(params=Multi_CNN.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-5) # Optimizer 
scheduler_multi = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_multi, factor=0.1, patience=2, verbose=True) # Scheduler

# Architecture definition
CNN_arch_multi = CNN_Architecture(model = Multi_CNN, 
             train_dataloader = train_dl_single,
             val_dataloader = val_dl_single,
             optimizer = optimizer_multi,
             loss_fn = nn.CrossEntropyLoss(weight=weights),
             score_fn = accuracy_score,
             scheduler = scheduler_multi,
             device = device,
             save_check = True,
             load_check_train = True,
             load_check_evaluate = False)

In [ ]:
experiments_results.append(train_evaluate(CNN_arch_multi, test_dl_single)) # Run train and evaluation

In [ ]:
test_single_image(test_data_single, test_dict, CNN_arch_multi) # Test in a single random image

## Multi Frame Model Late Fusion CNN

In [ ]:
AN_Late1, AN_late2 = AlexNet(in_channels=3), AlexNet(in_channels=3) # Extractors
Late_CNN = LateFusionCNN(AN_Late1, AN_late2, num_classes=n_classes) # CNN model

optimizer_late = torch.optim.SGD(params=Late_CNN.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-5) # Optimizer 
scheduler_late = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_late, factor=0.1, patience=2, verbose=True) # Scheduler

# Architecture definition
CNN_arch_late = CNN_Architecture(model = Late_CNN, 
             train_dataloader = train_dl_late,
             val_dataloader = val_dl_late,
             optimizer = optimizer_late,
             loss_fn = nn.CrossEntropyLoss(weight=weights),
             score_fn = accuracy_score,
             scheduler = scheduler_late,
             device = device,
             save_check = True,
             load_check_train = True,
             load_check_evaluate = False)

In [ ]:
experiments_results.append(train_evaluate(CNN_arch_late, test_dl_late)) # Run train and evaluation

In [ ]:
test_single_image(test_data_late, test_dict, CNN_arch_late) # Test in a single random image

## Multi Frame Model Early Fusion CNN

In [ ]:
AN_Early = AlexNet(in_channels=3, t_frames=[5,1,1]) # Extractor
Early_CNN = EarlyFusionCNN(AN_Early, num_classes=n_classes) # CNN model

optimizer_early = torch.optim.SGD(params=Early_CNN.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-5) # Optimizer 
scheduler_early = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_early, factor=0.1, patience=2, verbose=True) # Scheduler

# Architecture definition
CNN_arch_early = CNN_Architecture(model = Early_CNN, 
             train_dataloader = train_dl_early,
             val_dataloader = val_dl_early,
             optimizer = optimizer_early,
             loss_fn = nn.CrossEntropyLoss(weight=weights),
             score_fn = accuracy_score,
             scheduler = scheduler_early,
             device = device,
             save_check = True,
             load_check_train = True,
             load_check_evaluate = False)

In [ ]:
experiments_results.append(train_evaluate(CNN_arch_early, test_dl_early)) # Run train and evaluation

In [ ]:
test_single_image(test_data_early, test_dict, CNN_arch_early) # Test in a single random image

## Multi Frame Model Slow Fusion CNN

In [ ]:
AN_Slow = AlexNet(in_channels=3, t_frames=[4,2,2]) # Extractor
Slow_CNN = SlowFusionCNN(AN_Slow, num_classes=n_classes) # CNN model

optimizer_slow = torch.optim.SGD(params=Slow_CNN.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-5) # Optimizer 
scheduler_slow = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_slow, factor=0.1, patience=2, verbose=True) # Scheduler

# Architecture definition
CNN_arch_slow = CNN_Architecture(model = Slow_CNN, 
             train_dataloader = train_dl_slow,
             val_dataloader = val_dl_slow,
             optimizer = optimizer_slow,
             loss_fn = nn.CrossEntropyLoss(weight=weights),
             score_fn = accuracy_score,
             scheduler = scheduler_slow,
             device = device,
             save_check = True,
             load_check_train = True,
             load_check_evaluate = False)

In [ ]:
experiments_results.append(train_evaluate(CNN_arch_slow, test_dl_slow)) # Run train and evaluation

In [ ]:
test_single_image(test_data_slow, test_dict, CNN_arch_slow) # Test in a single random image

## Result Summary

Summary Widget with all the previous loss & score plot

In [ ]:
if (len(experiments_results) > 0):
    tb = widgets.TabBar([exp[0]['model_name'] for exp in experiments_results])

    for idx, exp in enumerate(experiments_results):
        history, _ = exp
        with tb.output_to(history['model_name'], select=(idx == 0)):
            plot_loss_curves(history)

Summary of test results

In [ ]:
if (len(experiments_results) > 0):
    test_results_grid = widgets.Grid(len(experiments_results) + 1, 3, header_row=True)

    with test_results_grid.output_to(0, 0): print('Model Name')
    with test_results_grid.output_to(0, 1): print('Loss')
    with test_results_grid.output_to(0, 2): print('Accuracy Score')

    for row, exp in enumerate(experiments_results):
        _, results = exp
        for col, val in enumerate(results.values()):
            with test_results_grid.output_to(row + 1, col): print(val)